## PDP / Forest Logging Detection

In [1]:
import ee
import geemap
import geopandas as gpd

#### testowanie

In [2]:
ee.Authenticate() 
ee.Initialize()

In [26]:
def visualize_forest_change(year):
    if year < 2001 or year > 2023:
        raise ValueError("Podaj rok w zakresie od 2001 do 2023.")
    ee.Initialize()


    poland = ee.FeatureCollection('FAO/GAUL/2015/level0') \
        .filter(ee.Filter.eq('ADM0_NAME', 'Poland'))

    hansen_data = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')
    
    loss = hansen_data.select('loss')
    loss_year = hansen_data.select('lossyear')
    # Filtr strat lasu dla wybranego roku
    loss_in_year = loss.updateMask(loss_year.eq(year - 2000))
    loss_in_year_poland = loss_in_year.clip(poland)
    
    satellite_data_following_year = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(f'{year + 1}-06-01', f'{year + 1}-08-31') \
        .filterBounds(poland) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .median() \
        .clip(poland)

    satellite_data_previous_year = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(f'{year - 1}-06-01', f'{year - 1}-08-31') \
        .filterBounds(poland) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .median() \
        .clip(poland)
    
    # ustalenie obszaru zalesionego
    forest_cover = hansen_data \
        .select('treecover2000') \
        .clip(poland) \
        .gte(25) \
        .selfMask()
        # .And(
        #     hansen_data \
        #         .select('lossyear') \
        #         .gt(year - 2000) \
        #         .Or(hansen_data.select('lossyear').eq(0))
      # )
    

    loss_viz = {
        'min': 0,
        'max': 1,
        'palette': ['red']
    }
    satellite_viz = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.2
    }

    map = geemap.Map()
    map.centerObject(poland, 6)  # Przybliżenie na Polskę
    map.addLayer(satellite_data_following_year, satellite_viz, f'SAT {year + 1}')
    map.addLayer(satellite_data_previous_year, satellite_viz, f'SAT {year - 1}')
    map.addLayer(loss_in_year_poland, loss_viz, f'Straty lasu w {year}')
    map.addLayer(poland, {'color': 'blue'}, 'Granice Polski')
    map.addLayer(forest_cover, {'palette': ['green']}, 'Obszary leśne')

    return map



In [27]:
visualize_forest_change(2022)

Map(center=[52.108736119738694, 19.430639807941635], controls=(WidgetControl(options=['position', 'transparent…

In [86]:
def export_forest_loss_to_geojson(year, output_file):
    if year < 2001 or year > 2023:
        raise ValueError("Podaj rok w zakresie od 2001 do 2023.")

    ee.Initialize()

    poland = ee.FeatureCollection('FAO/GAUL/2015/level0') \
        .filter(ee.Filter.eq('ADM0_NAME', 'Poland'))
    poland_geometry = poland.geometry()

    hansen_data = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

    #Straty lasu
    loss = hansen_data.select('loss')
    loss_year = hansen_data.select('lossyear')


    loss_in_year = loss.updateMask(loss_year.eq(year - 2000))
    loss_in_year_poland = loss_in_year.clip(poland_geometry)


    loss_vector = loss_in_year_poland.reduceToVectors(
        geometry=poland_geometry,
        geometryType='polygon',
        reducer=ee.Reducer.countEvery(),
        scale=30,
        maxPixels=1e9, 
        bestEffort=True 
    )

    geemap.ee_export_vector(
        ee_object=loss_vector,
        filename=output_file
    )
export_forest_loss_to_geojson(2020, 'forest_loss_2020.geojson')


KeyboardInterrupt: 

In [98]:
geemap.basemaps.keys()

dict_keys(['OpenStreetMap', 'Esri.WorldStreetMap', 'Esri.WorldImagery', 'Esri.WorldTopoMap', 'FWS NWI Wetlands', 'FWS NWI Wetlands Raster', 'NLCD 2021 CONUS Land Cover', 'NLCD 2019 CONUS Land Cover', 'NLCD 2016 CONUS Land Cover', 'NLCD 2013 CONUS Land Cover', 'NLCD 2011 CONUS Land Cover', 'NLCD 2008 CONUS Land Cover', 'NLCD 2006 CONUS Land Cover', 'NLCD 2004 CONUS Land Cover', 'NLCD 2001 CONUS Land Cover', 'USGS NAIP Imagery', 'USGS NAIP Imagery False Color', 'USGS NAIP Imagery NDVI', 'USGS Hydrography', 'USGS 3DEP Elevation', 'ESA Worldcover 2020', 'ESA Worldcover 2020 S2 FCC', 'ESA Worldcover 2020 S2 TCC', 'ESA Worldcover 2021', 'ESA Worldcover 2021 S2 FCC', 'ESA Worldcover 2021 S2 TCC', 'BaseMapDE.Color', 'BaseMapDE.Grey', 'BasemapAT.basemap', 'BasemapAT.grau', 'BasemapAT.highdpi', 'BasemapAT.orthofoto', 'BasemapAT.overlay', 'BasemapAT.surface', 'BasemapAT.terrain', 'CartoDB.DarkMatter', 'CartoDB.DarkMatterNoLabels', 'CartoDB.DarkMatterOnlyLabels', 'CartoDB.Positron', 'CartoDB.Posit

In [28]:
import geemap
import geopandas as gpd
import ee

ee.Initialize()

gdf = gpd.read_file('forest_loss_2020.geojson')

if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)


first_sample = gdf.iloc[[4]]


ee_object = geemap.geopandas_to_ee(first_sample)
poland = ee.FeatureCollection('FAO/GAUL/2015/level0') \
        .filter(ee.Filter.eq('ADM0_NAME', 'Poland'))


vis_params = {
    'min': 0,
    'max': 3000,
}

map = geemap.Map()
map.setOptions('SATELLITE')
map.centerObject(ee_object, 15)  



map.addLayer(ee_object, {'color': 'red'}, 'Pierwsza próbka')

map


Map(center=[52.94717233404858, 14.170007240720212], controls=(WidgetControl(options=['position', 'transparent_…